In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import os
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from hcrot import layers, optim
from hcrot.dataset import *
from hcrot.utils import *

In [3]:
lr_rate = 1e-3
hidden_size = 256
epochs = 10

In [4]:
df = pd.read_csv("./datasets/mnist_test.csv")
label = df['7'].to_numpy()
df = df.drop('7', axis=1)
dat = (df / 255.).to_numpy()

dataset_len = len(dat)
dat = dat.reshape(dataset_len, 28, 28).astype(np.float32)

In [5]:
train_image, test_image = dat[:5000], dat[8001:9001]
train_label, test_label = label[:5000], label[8001:9001]

In [6]:
dataloader = Dataloader(train_image, train_label, batch_size=50, shuffle=True)
testloader = Dataloader(test_image, test_label, batch_size=1, shuffle=False)

### RNN

In [ ]:
class RNN(layers.Module):
    def __init__(self, hidden_size=32, num_classes=10):
        super().__init__()
        self.rnn = layers.RNN(
            input_size=28,
            hidden_size=hidden_size,
            num_layers=1,
            nonlinearity='tanh', # or 'relu'
            batch_first=True
        )
        self.fc = layers.Linear(hidden_size, num_classes)

    def forward(self, x):
        x, _ = self.rnn(x)
        return self.fc(x[:,-1,:])

In [8]:
model = RNN(hidden_size=hidden_size, num_classes=10)
criterion = layers.CrossEntropyLoss()
optimizer = optim.Adam(model, lr_rate=lr_rate)

In [9]:
for epoch in range(epochs):
    total_loss, correct = 0, 0

    model.train()
    for images, labels in dataloader:
        logits = model(images)
        loss = criterion(logits, labels)
        
        total_loss += loss.item()
        dz = criterion.backward()
        optimizer.update(dz)

    model.eval()
    for images, labels in testloader:
        logits = model(images)
        predictions = np.argmax(logits, axis=1)
        correct += (predictions == labels).item()

    print(f'Epoch [{epoch+1} / {epochs}] | Loss: {total_loss / len(dataloader):.3f} | Acc: {correct / len(testloader):.3f}')

Epoch [1 / 10] | Loss: 1.446 | Acc: 0.702
Epoch [2 / 10] | Loss: 0.908 | Acc: 0.772
Epoch [3 / 10] | Loss: 0.785 | Acc: 0.802
Epoch [4 / 10] | Loss: 0.708 | Acc: 0.848
Epoch [5 / 10] | Loss: 0.622 | Acc: 0.806
Epoch [6 / 10] | Loss: 0.558 | Acc: 0.816
Epoch [7 / 10] | Loss: 0.470 | Acc: 0.817
Epoch [8 / 10] | Loss: 0.472 | Acc: 0.873
Epoch [9 / 10] | Loss: 0.459 | Acc: 0.820
Epoch [10 / 10] | Loss: 0.381 | Acc: 0.906


### LSTM

In [10]:
class LSTM(layers.Module):
    def __init__(self, hidden_size=32, num_classes=10):
        super().__init__()
        self.lstm = layers.LSTM(
            input_size=28,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )
        self.fc = layers.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x, _, _ = self.lstm(x)
        return self.fc(x[:,-1,:])

In [11]:
model = LSTM(hidden_size=hidden_size, num_classes=10)
criterion = layers.CrossEntropyLoss()
optimizer = optim.Adam(model, lr_rate=lr_rate)

In [12]:
for epoch in range(epochs):
    total_loss, correct = 0, 0

    model.train()
    for images, labels in dataloader:
        logits = model(images)
        loss = criterion(logits, labels)
        
        total_loss += loss.item()
        dz = criterion.backward()
        optimizer.update(dz)

    model.eval()
    for images, labels in testloader:
        logits = model(images)
        predictions = np.argmax(logits, axis=1)
        correct += (predictions == labels).item()

    print(f'Epoch [{epoch+1} / {epochs}] | Loss: {total_loss / len(dataloader):.3f} | Acc: {correct / len(testloader):.3f}')

Epoch [1 / 10] | Loss: 1.554 | Acc: 0.685
Epoch [2 / 10] | Loss: 0.861 | Acc: 0.780
Epoch [3 / 10] | Loss: 0.649 | Acc: 0.851
Epoch [4 / 10] | Loss: 0.482 | Acc: 0.913
Epoch [5 / 10] | Loss: 0.356 | Acc: 0.953
Epoch [6 / 10] | Loss: 0.266 | Acc: 0.951
Epoch [7 / 10] | Loss: 0.217 | Acc: 0.914
Epoch [8 / 10] | Loss: 0.215 | Acc: 0.924
Epoch [9 / 10] | Loss: 0.165 | Acc: 0.932
Epoch [10 / 10] | Loss: 0.147 | Acc: 0.939
